In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [2]:
# Load data
data = pd.read_csv('../01_Data/data_california_house_prepared.csv')

In [ ]:
# Membuat fitur baru
data['rooms_per_household'] = data['total_rooms']/data['households']
data['bedrooms_per_room'] = data['total_bedrooms']/data['total_rooms']
data['population_per_household'] = data['population']/data['households']

# Memeriksa korelasi fitur baru dengan target
new_features = ['rooms_per_household', 'bedrooms_per_room', 'population_per_household']
print(data[new_features + ['median_house_value']].corr()['median_house_value'])

rooms_per_household         0.159258
bedrooms_per_room          -0.253594
population_per_household   -0.030762
median_house_value          1.000000
Name: median_house_value, dtype: float64


In [4]:
# Pisahkan fitur dan target
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline
numeric_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                    'total_bedrooms', 'population', 'households', 'median_income', 'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical_features = ['ocean_proximity']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [5]:
# Daftar model
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)
}

# Evaluasi setiap model
for name, regressor in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', regressor)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"{name}: R-squared = {score:.3f}")

Linear Regression: R-squared = 0.637
Ridge Regression: R-squared = 0.638


d:\PurwadhikaBackup\capstoneproject\modultiga\california_housing_price_prediction\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.538e+12, tolerance: 1.537e+10
  model = cd_fast.enet_coordinate_descent(


Lasso Regression: R-squared = 0.637
Decision Tree: R-squared = 0.617
Random Forest: R-squared = 0.807
XGBoost: R-squared = 0.830
